# Network Deliberation ABM
## Discrete state, generated NK Model task

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches': None}

# Imports
import configparser
import itertools
import math

import matplotlib as mpl
from matplotlib import pyplot as plt
import netdelib.soclearn as slearn
import numpy as np
import pandas as pd
import repsci
import scipy.stats

# Configure plotting in Jupyter
from matplotlib import pyplot as plt

plt.rcParams['font.size'] = 18
plt.rcParams['axes.linewidth'] = 2

symbols = ['.','.','.','.']
colors = ['#0000ff','#999966','#dddd77', '#aaaaff']
plt.figure(figsize=(7.5,10))
exclude = ['Random', 'StochasticBlock', 'Lattice']

In [ ]:
def load_results (experiment_id):
    
    experiment = repsci.Experiment(
        "discrete_generated_nk",
        reproduce=experiment_id)
    config = experiment.get_config()
    
    score = np.load(experiment.get_filename('score_run.npy'))
    correct = np.load(experiment.get_filename('correct_run.npy'))
    distance_score = np.load(experiment.get_filename('distance_score_run.npy'))
    # Older code did not produce the below data
    try:
        initial_distance = np.load(experiment.get_filename('initial_distance_run.npy'))
    except FileNotFoundError:
        initial_distance = None
    try:
        converge_time = np.load(experiment.get_filename('converge_time_run.npy'))
    except FileNotFoundError:
        converge_time = None
    return experiment, score, correct, distance_score, initial_distance, converge_time

def individual_title(config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
#        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
#            title_individual = 'Hill-Climb'
#        else:
#            title_individual = 'Mutation'
        title_individual = ''
        if individual_mode == slearn.MODE_ALL:
            title_individual += 'Serial-Indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += 'Fallback-Indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += 'Parallel-Indiv.'
    else:
        title_individual = 'social-only'
    return title_individual

def result_title (config):
    individual_mode = config.getint('strategies', 'individual_mode', fallback=1)
    if config.getboolean('strategies', 'individual', fallback=False):
        if config.getboolean('strategies', 'individual_all_bits', fallback=True):
            title_individual = 'hill-climb'
        else:
            title_individual = 'mutation'
        if individual_mode == slearn.MODE_ALL:
            title_individual += ' serial indiv.'
        elif individual_mode == slearn.MODE_FALLBACK:
            title_individual += ' fallback indiv.'
        elif individual_mode == slearn.MODE_BEST:
            title_individual += ' parallel indiv.'
    else:
        title_individual = 'social-only'
    title = "|V|={} N={}, K={}, EXP={}, S={}, {}, {}".format(
        config.getint('abm', 'N'),
        config.getint('abm', 'bit_count'),
        config.getint('abm', 'K'),
        config.getfloat('abm', 'nk_exponent', fallback=1),
        config.getint('strategies', 'sample', fallback=None),
        {True: 'critical', False: 'non-critical'}[config.getboolean('strategies', 'critical')],
        title_individual)
    return title

In [ ]:
experiment, score, correct, distance_score, initial_distance, converge_time = load_results(
    '2021-02-04 161927 86fe505')
config = experiment.config

networks = experiment.config.get('networks', 'enabled').split(',')
learning_strategies = experiment.config.get('strategies', 'enabled').split(',')

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

# All possible combinations
combos = list(itertools.product(['fallback'], range(len(learning_strategies)), range(len(networks))))

experiment, score, correct, distance_score, initial_distance, converge_time = load_results(
    '2021-02-08 133817 86fe505')
config = experiment.config

combos += list(itertools.product(['parallel'], range(len(learning_strategies)), range(len(networks))))

keys = []

result = {}
for i, row in enumerate(combos):
    
    row_strat_idx = row[1]
    row_net_idx = row[2]
    row_hybrid = row[0]
    row_strat = learning_strategies[row_strat_idx]
    row_net = networks[row_net_idx]
    row_name = '{}:{}:{}'.format(row_hybrid, row_strat, row_net)

    result_cells = {}
    
    for j, col in enumerate(combos):
        
        col_strat_idx = col[1]
        col_net_idx = col[2]        
        col_hybrid = col[0]
        col_strat = learning_strategies[col_strat_idx]
        col_net = networks[col_net_idx]
        
        if row_net in exclude or col_net in exclude:
            continue
            
        if row_strat == 'Local Majority' or col_strat == 'Local Majority':
            continue
        
        a = [max(run_score) for run_score in score[col_strat_idx][col_net_idx]]
        b = [max(run_score) for run_score in score[row_strat_idx][row_net_idx]]
        differences = [
            a[r] - b[r]
            for r in range(runs)
        ]
        
        col_name = '{}:{}:{}'.format(col_hybrid, col_strat, col_net)
        if col_name not in keys:
            keys.append(col_name)
        
        dbar = np.mean(differences)
        T, p = scipy.stats.ttest_rel(a, b)
        
        cell = (dbar,p)
        result_cells[col_name] = cell
    result[row_name] = result_cells

with open(experiment.get_filename('ttest.tsv'), 'w') as f:
    
    header_row = '\t'.join(['', 'a', 'b', 'b-a', 'p'])
    f.write(header_row + '\n')

    for a in keys:
        for b in keys:
            if a == b:
                continue
            f.write('\t'.join([a, b, str(result[a][b][0]), str(result[a][b][1])]) + '\n')


In [ ]:
keys

In [ ]:
experiment, score, correct, distance_score, initial_distance, converge_time = load_results(
    '2021-02-04 161927 86fe505')
config = experiment.config

networks = experiment.config.get('networks', 'enabled').split(',')
learning_strategies = experiment.config.get('strategies', 'enabled').split(',')

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

# All possible combinations
combos = list(itertools.product(['fallback'], range(len(learning_strategies)), range(len(networks))))

results = []

for strat_idx in range(len(learning_strategies)):    
    
    strat = learning_strategies[strat_idx]

    if strat == 'Local Majority':
        continue
        
    for net_a_idx in range(len(networks)):
        net_a = networks[net_a_idx]

        for net_b_idx in range(len(networks)):
            net_b = networks[net_b_idx]

            if net_a in exclude or net_b in exclude or net_a == net_b:
                continue
            
            a = [max(run_score) for run_score in score[strat_idx][net_a_idx]]
            b = [max(run_score) for run_score in score[strat_idx][net_b_idx]]
            differences = [
                a[r] - b[r]
                for r in range(runs)
            ]

            dbar = np.mean(differences)
            T, p = scipy.stats.ttest_rel(a, b)
        
            results.append( (strat, net_a, net_b, '{:0.2e}'.format(dbar), '{:0.2e}'.format(T), '{:0.2e}'.format(60 * p)) )

with open(experiment.get_filename('ttest-fallback-instrat.txt'), 'w') as f:
    
    header_row = '&'.join(['strategy', 'net a', 'net b', 'b-a', 'T', 'p*'])
    f.write(header_row + '\\\\\n')

    for row in results:
        f.write('&'.join(row) + '\\\\\n')


In [ ]:
experiment, score, correct, distance_score, initial_distance, converge_time = load_results(
    '2021-02-04 161927 86fe505')
config = experiment.config

networks = experiment.config.get('networks', 'enabled').split(',')
learning_strategies = experiment.config.get('strategies', 'enabled').split(',')

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

# All possible combinations
combos = list(itertools.product(['fallback'], range(len(learning_strategies)), range(len(networks))))

results = []

for net_idx in range(len(networks)):    
    
    net = networks[net_idx]

    if net in exclude:
        continue

    for strat_a_idx in range(len(learning_strategies)):
        strat_a = learning_strategies[strat_a_idx]

        if strat_a == 'Local Majority':
            continue

        for strat_b_idx in range(len(learning_strategies)):
            strat_b = learning_strategies[strat_b_idx]

            if strat_b == 'Local Majority' or strat_a == strat_b:
                continue
            
            a = [max(run_score) for run_score in score[strat_a_idx][net_idx]]
            b = [max(run_score) for run_score in score[strat_b_idx][net_idx]]
            differences = [
                a[r] - b[r]
                for r in range(runs)
            ]

            dbar = np.mean(differences)
            T, p = scipy.stats.ttest_rel(a, b)
        
            results.append( (net, strat_a, strat_b, '{:0.2e}'.format(dbar), '{:0.2e}'.format(T), '{:0.2e}'.format(60 * p)) )

with open(experiment.get_filename('ttest-fallback-innet.txt'), 'w') as f:
    
    header_row = '&'.join(['net', 'strategy a', 'strategy b', 'b-a', 'T', 'p*'])
    f.write(header_row + '\\\\\n')

    for row in results:
        f.write('&'.join(row) + '\\\\\n')


In [ ]:
experiment, score, correct, distance_score, initial_distance, converge_time = load_results(
    '2021-02-08 133817 86fe505')
config = experiment.config

networks = experiment.config.get('networks', 'enabled').split(',')
learning_strategies = experiment.config.get('strategies', 'enabled').split(',')

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

# All possible combinations
combos = list(itertools.product(['fallback'], range(len(learning_strategies)), range(len(networks))))

results = []

for strat_idx in range(len(learning_strategies)):    
    
    strat = learning_strategies[strat_idx]

    if strat == 'Local Majority':
        continue
        
    for net_a_idx in range(len(networks)):
        net_a = networks[net_a_idx]

        for net_b_idx in range(len(networks)):
            net_b = networks[net_b_idx]

            if net_a in exclude or net_b in exclude or net_a == net_b:
                continue
            
            a = [max(run_score) for run_score in score[strat_idx][net_a_idx]]
            b = [max(run_score) for run_score in score[strat_idx][net_b_idx]]
            differences = [
                a[r] - b[r]
                for r in range(runs)
            ]

            dbar = np.mean(differences)
            T, p = scipy.stats.ttest_rel(a, b)
        
            results.append( (strat, net_a, net_b, '{:0.2e}'.format(dbar), '{:0.2e}'.format(T), '{:0.2e}'.format(60 * p)) )

with open(experiment.get_filename('ttest-parallel-instrat.txt'), 'w') as f:
    
    header_row = '&'.join(['strategy', 'net a', 'net b', 'b-a', 'T', 'p*'])
    f.write(header_row + '\\\\\n')

    for row in results:
        f.write('&'.join(row) + '\\\\\n')


In [ ]:
experiment, score, correct, distance_score, initial_distance, converge_time = load_results(
    '2021-02-08 133817 86fe505')
config = experiment.config

networks = experiment.config.get('networks', 'enabled').split(',')
learning_strategies = experiment.config.get('strategies', 'enabled').split(',')

num_net = len(networks)
num_strat = len(learning_strategies)
runs = config.getint('abm', 'runs')

# All possible combinations
combos = list(itertools.product(['fallback'], range(len(learning_strategies)), range(len(networks))))

results = []

for net_idx in range(len(networks)):    
    
    net = networks[net_idx]

    if net in exclude:
        continue

    for strat_a_idx in range(len(learning_strategies)):
        strat_a = learning_strategies[strat_a_idx]

        if strat_a == 'Local Majority':
            continue

        for strat_b_idx in range(len(learning_strategies)):
            strat_b = learning_strategies[strat_b_idx]

            if strat_b == 'Local Majority' or strat_a == strat_b:
                continue
            
            a = [max(run_score) for run_score in score[strat_a_idx][net_idx]]
            b = [max(run_score) for run_score in score[strat_b_idx][net_idx]]
            differences = [
                a[r] - b[r]
                for r in range(runs)
            ]

            dbar = np.mean(differences)
            T, p = scipy.stats.ttest_rel(a, b)
        
            results.append( (net, strat_a, strat_b, '{:0.2e}'.format(dbar), '{:0.2e}'.format(T), '{:0.2e}'.format(60 * p)) )

with open(experiment.get_filename('ttest-parallel-innet.txt'), 'w') as f:
    
    header_row = '&'.join(['net', 'strategy a', 'strategy b', 'b-a', 'T', 'p*'])
    f.write(header_row + '\\\\\n')

    for row in results:
        f.write('&'.join(row) + '\\\\\n')
